# Modelbuilder

This notebook illustrates the set up of a model to investigate NbS for coastal flooding. The example here presents a usecase for the Wash (United Kingdom) and combines input data from multiple sources. The model is set up using the HydroMT_sfincs python package. The usecase here requires a selection of its functionalities. If you want to learn more about building sfincs models this way, visit [HydroMT_sfincs - Modelbuilder example from script](https://github.com/Deltares/hydromt_sfincs/blob/main/examples/build_from_script.ipynb).

This modelbuilder combines many other Python packages. HydroMT_sfincs is used for the set up of the sfincs model, matplotlib for visualization, (rio)xarray to process netcdf files and geopandas to process geospatial data.



## Imports and user variables

In [ ]:
from shapely.geometry import Point
from os.path import join
import geopandas as gpd
import hydromt
from hydromt_sfincs import SfincsModel
import os
import shutil

In [ ]:
# Define the main directory and data library
# This is the directory where the data library is stored
data_libs = './input_dir/edito_sfincs_data.yml'
input_dir = './input_dir' 
main_dir = './wash_uk_example' 

# This is the directory where the model will be saved
sim_dir   = join(main_dir, 'sims')
if not os.path.exists(sim_dir):
    os.makedirs(sim_dir)

In [ ]:
# Define the input files
fn_domain       =  join(input_dir, 'domain.gpkg')
fn_domain_lines =  join(input_dir, 'domain_lines.gpkg')
fn_topo         = join(input_dir, 'delta_dtm_gebco_ref_msl.tif')
fn_veg          =  join(input_dir, 'da_veg.tif')
fn_storm_wl     =  join(input_dir, 'wl_ts.nc')

In [4]:
# Define the grid resolution
grid_resolution     = 50     # meters

# Define the roughness parameters
manning_saltmarshes = 0.08   # 0.07 Rezaie et al., 2020
manning_mangroves   = 0.15   # Zhang et al., 2012
manning_land        = 0.04   # default
manning_sea         = 0.02   # default

# Define waterlevel offset
slr_mean = 0.0

In [5]:
# Define the model observation points
obs_names             = ['veg1','veg2']
obs_pnts              = [Point(303082.833,5864791.153), 
                         Point(315739.571,5854677.962)]

## Initialization

In [ ]:
# Initialize hydromt DataCatalog
data_cat  = hydromt.DataCatalog(data_libs = data_libs)

# Initialize SFINCS model
mod = SfincsModel(root  = sim_dir+ '/sim_veg',  mode = 'w+', data_libs = data_libs)
gdf_domain       = gpd.read_file(fn_domain)
gdf_domain_lines = gpd.read_file(fn_domain_lines) 

Model dir already exists and files might be overwritten: p:\11209182-edito\06-Models\SFINCS_nbs_example\03_wash_uk_example\sims_test\sim_veg\gis.


## Grid & Topography

In [7]:
# setup grid from polygon in GeoDataFrame   
mod.setup_grid_from_region(region = {'geom':gdf_domain}, res = grid_resolution, rotated = True)
mod.config

# Define topography and vegetation datasets
da_topo   = data_cat.get_rasterdataset(fn_topo)
da_veg    = data_cat.get_rasterdataset(fn_veg)

# Assign topography to the model
datasets_dep  = [{"elevtn":da_topo}]
da_dep        = mod.setup_dep(datasets_dep=datasets_dep, buffer_cells= 1) # 

# set mask based on elevation (areas in km2)
mod.setup_mask_active(mask = gdf_domain , zmax = 10, drop_area = 10, fill_area = 100, reset_mask = True) 
mod.set_config("stopdepth", '250.0')
   
# Assign observation points to the model
gdf_obs               = gpd.GeoDataFrame({'names': obs_names}, geometry = obs_pnts, crs = mod.crs).set_index('names')
mod.setup_observation_points(gdf_obs)

## Boundary Conditions

In [8]:
# Define model boundaries
gdf_all_lines = gdf_domain_lines.loc[(gdf_domain_lines['bnd']==1)] 
if len(gdf_all_lines) > 0:
    gdf_all_lines = gdf_all_lines.loc[gdf_all_lines['geometry'] != None]
    gdf_all_lines.geometry = gdf_all_lines['geometry'].buffer(0.0075)
mod.setup_mask_bounds(btype = "waterlevel", include_mask = gdf_all_lines  , reset_bounds=True, zmax = 1) # here we mask the wl for all bnd points     

#update times, in line with wl forcing (times should be updated afeter setup_mask_bounds)
mod.set_config("tref"  , "20000101 000000")
mod.set_config("tstart", "20000101 000000")
mod.set_config("tstop" , "20000106 000000")
mod.set_config("dtout",  "600")


## Forcing

In [ ]:

mod.setup_waterlevel_forcing(geodataset = fn_storm_wl, offset=slr_mean , buffer = 15000)
df_ts = mod.forcing['bzs'].to_dataframe()
df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[0]
df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[-1]
t_start = str(df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[0][0]).replace('-','').replace(':','')
t_end   = str(df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[-1][0]).replace('-','').replace(':','')

#update times, in line with wl forcing
mod.set_config("tref"  , "20000101 000000")
mod.set_config("tstart", t_start)
mod.set_config("tstop" , t_end)
mod.set_config("dtout",  "600")
mod.set_config("dtmaxout","9999999")

## Add spatially varying roughness data

To incorporate the influence of different roughness for land, sea, and vegetation on flow this is added to the model based on our data.

In [11]:
# set increased Manning values for vegetation cover
mapping = {1:manning_saltmarshes, 2:manning_mangroves}
manning_veg_map = da_veg.copy()
for old_value, new_value in mapping.items():
    manning_veg_map = manning_veg_map.where(manning_veg_map != old_value, new_value)
  
datasets_rgh = [{'manning':manning_veg_map}]
mod.setup_manning_roughness(
           datasets_rgh=datasets_rgh,
           manning_land= manning_land,
           manning_sea= manning_sea,
           rgh_lev_land=0,  # the minimum elevation of the land
       )

## Write to file

In [ ]:
# Write the model with vegation to files
mod.write()

# create a copy for the model without vegetation for comparison
shutil.copytree(sim_dir + '/sim_veg', sim_dir + '/sim_noveg', dirs_exist_ok = True)
os.remove(sim_dir + '/sim_noveg/hydromt_data.yml')

# load the model and change manning file 
mod_noveg = SfincsModel(root  = sim_dir + '/sim_noveg',  mode = 'r+', data_libs = data_libs)
mod_noveg.config

mod_noveg.setup_manning_roughness(
              manning_land = manning_land,
              manning_sea  = manning_sea,
              rgh_lev_land = 0)  # the minimum elevation of the land

mod_noveg.write()

## Upload model to EDITO storage [no vegetation]

In [ ]:
# upload the generated model to youe s3 bucket in "SFINCS_INPUT"
from upload_model import upload_model_to_s3_bucket
upload_model_to_s3_bucket(join(sim_dir, 'sim_noveg'), 'SFINCS_INPUT')

## Run SFINCS simulation [no vegetation]

In [ ]:
from launch_sfincs_process import launch_process_in_browser
launch_process_in_browser(
        cpu_request="1800m",
    memory_request="5Gi",
    cpu_limit="6400m",
    memory_limit="28Gi"
)

## Import simulation output [no vegetation]

In [ ]:
from download_from_s3 import download_nc_files_from_s3

# import simulation output
download_nc_files_from_s3(
    files_to_download=['sfincs_map.nc', 'sfincs_his.nc'],
    s3_subfolder='SFINCS_OUTPUT',
    local_output_dir=join(sim_dir, 'sim_noveg')  # or 'sim_veg'
)

## Upload model to EDITO storage [vegetation]

In [ ]:
# upload the generated model to youe s3 bucket in "SFINCS_INPUT"
from upload_model import upload_model_to_s3_bucket
upload_model_to_s3_bucket(join(sim_dir, 'sim_veg'), 'SFINCS_INPUT')

## Run SFINCS simulation [vegetation]

In [ ]:
from launch_sfincs_process import launch_process_in_browser
launch_process_in_browser(
        cpu_request="1800m",
    memory_request="5Gi",
    cpu_limit="6400m",
    memory_limit="28Gi"
)

Import simulation output [vegetation]

In [ ]:
from download_from_s3 import download_nc_files_from_s3

# import simulation output
download_nc_files_from_s3(
    files_to_download=['sfincs_map.nc', 'sfincs_his.nc'],
    s3_subfolder='SFINCS_OUTPUT',
    local_output_dir=join(sim_dir, 'sim_veg')  # or 'sim_veg'
)

## Visualize output

In [ ]:
def sfincs_mod_read_h(dir_sfincs_model, hmin=0.05, apply_hmin = False):
    
    mod = SfincsModel(dir_sfincs_model, mode = "r")
    
    mod.read_results(chunksize = 10)
    # the following variables have been found
    list(mod.results.keys())
    da_h = mod.results["h"]
    if apply_hmin == True:
        da_h = da_h.where(da_h > hmin)
    da_h.attrs.update(long_name="flood depth", unit="m")
    
    return da_h

def load_hmax_noveg(dir_sfincs_model_noveg, dir_sfincs_model_veg, hmin = 0.05):
    # load model
    da_h_noveg = sfincs_mod_read_h(dir_sfincs_model_noveg, hmin)
    da_h_veg   = sfincs_mod_read_h(dir_sfincs_model_veg, hmin)
    dif = da_h_veg - da_h_noveg # postive values are increase, negative equals reduction
    
    # veg extent
    mod = SfincsModel(dir_sfincs_model_noveg, mode = "r")

    return dif, da_h_noveg, mod

def load_h_his(dir_sfincs_model_noveg, dir_sfincs_model_veg,
               obs = ''):
    mod_noveg = SfincsModel(dir_sfincs_model_noveg, mode = "r")
    mod_veg   = SfincsModel(dir_sfincs_model_veg, mode = "r") 
    
    if len(obs) ==0:
        station_id = 0 
        
    h_noveg = mod_noveg.results['point_h'].isel(stations = station_id)
    h_veg   = mod_veg.results['point_h'].isel(stations = station_id)

    return h_noveg, h_veg


def numfmt(x,pos):
    s = '{}'.format(x/1000.0)
    return s



def make_movie_from_pngs(folder, fps = 4):

    
    def update(frame):
        img.set_data(images[frame])
        return img


    #list files in folder
    png_files = [ f.path for f in os.scandir(folder) if f.is_file() ]
    
    images = [Image.open(path) for path in png_files]
    
    fig, ax = plt.subplots(figsize = (8,4), dpi = 300)
    ax.set_axis_off()  # Turn off axis for cleaner animation
    img = ax.imshow(images[0])  # Display the first image
    
    
    ani = animation.FuncAnimation(fig, update, frames=len(images), interval=200, blit=False)
    # save to file
    ani.save(join(folder,'output_movie.mp4'), writer='ffmpeg', fps = fps, dpi = 300)

In [ ]:
dir_sfincs_model_noveg =  join(sim_dir, 'sim_noveg')
dir_sfincs_model_veg =  join(sim_dir, 'sim_veg')

out_dir = join(sim_dir, 'figs/dif')
out_dir_his = join(sim_dir, 'figs')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

hmin     = 0.05
fs_ticks = 9
plot_veg = False
EO       = True
zoomlevel = 10
da_h_dif, da_h_noveg, mod = load_hmax_noveg(dir_sfincs_model_noveg, dir_sfincs_model_veg, hmin)
his_noveg, his_veg        = load_h_his(dir_sfincs_model_noveg, dir_sfincs_model_veg)

gswo       = data_cat.get_rasterdataset(join(input_dir, 'gswo.tif'))
gswo_mask  = gswo.raster.reproject_like(mod.grid, method="max") <= 5

da_h_noveg = da_h_noveg.where(gswo_mask).where(da_h_noveg > hmin)

# apply min dif filter
da_h_dif = da_h_dif.where(abs(da_h_dif)>hmin/2)

In [ ]:
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.io.img_tiles import GoogleTiles
import datetime
import matplotlib.ticker as tkr
from pylab import plot, show, savefig, xlim, figure, ylim, legend, boxplot, setp, axes
import gc

utm_zone = da_h_dif.raster.crs.to_wkt().split("UTM zone ")[1][:3]
extent = np.array(da_h_dif.raster.box.buffer(1e2).total_bounds)[[0, 2, 1, 3]]

for t in da_h_dif.time[:]:

    crs = ccrs.UTM(int(utm_zone[:2]))
    fig, axs = plt.subplots(1,2, subplot_kw={'projection': crs}, figsize = (8,4))

    for ax in axs:
        ax.set_extent(extent, crs = crs)
        tiler = GoogleTiles(style = 'satellite')
        ax.add_image(tiler, int(zoomlevel))

    ax1 = axs[0]; ax2 = axs[1]
    
    # plot difference veg no veg
    cbar_kwargs = {"shrink": 0.8, "orientation":'horizontal', "label":'Flood depth [m +MSL]'}
    da_plot = da_h_noveg.sel(time = t)
    cax_fld = da_plot.plot(
        x="xc", y="yc",
        ax=ax1,
        vmin=0, vmax=2,
        cmap=plt.cm.Spectral,
        alpha = 0.8,
        cbar_kwargs = cbar_kwargs)

    # plot difference veg no veg
    cbar_kwargs = {"shrink": 0.8, "orientation":'horizontal', "label":'$\Delta$ Flood depth [m]'}
    da_plot = da_h_dif.sel(time = t)
    cax_fld = da_plot.plot(
        x="xc", y="yc",
        ax=ax2,
        vmin=-0.5, vmax=0.5,
        cmap=plt.cm.bwr,
        alpha = 0.8,
        cbar_kwargs = cbar_kwargs)
    
    
    
    ax1.scatter(his_noveg.station_x.values, his_noveg.station_y.values, color = 'white', s = 5, marker = '*', zorder = 5)
    
    
    
    datesuffix       = str(np.datetime64(t.time.values,'m')).replace('-','');
    datesuffix_title = datetime.datetime.strptime(datesuffix, "%Y%m%dT%H:%M").strftime("%d-%m-%Y %H:%M")
    print(datesuffix)
    ax = [ax1,ax2]
    utm_suffix = da_h_dif.raster.crs.name.split('/')[1]
    for i in np.arange(2):
    # update axis labels
        ax[i].xaxis.set_visible(True)
        ax[i].yaxis.set_visible(True)
        ax[i].set_xlabel('X-coordinate [km] ' + utm_suffix, fontsize = fs_ticks)
        if i == 0: # ylabel on
            ax[i].set_ylabel('Y-coordinate [km] ' + utm_suffix, fontsize = fs_ticks)
        else:
            ax[i].set_ylabel('')
        yfmt = tkr.FuncFormatter(numfmt)
        ax[i].yaxis.set_major_formatter(yfmt); ax[i].xaxis.set_major_formatter(yfmt)
        ax[i].tick_params(axis="both", direction='out', length=3, labelsize = fs_ticks); 

        ax[i].set_title(datesuffix_title)


    savefig(join(out_dir, datesuffix.replace(':','_') + '_' + 'h_dif.png'), dpi = 300,bbox_inches='tight')

    fig.clf()
    plt.close(fig)
    gc.collect()

In [ ]:
# make movie
from PIL import Image
from matplotlib import animation
make_movie_from_pngs(out_dir, fps = 4)

In [ ]:
#Plot forcing
import xarray as xr
da_wl = xr.open_dataset(join(input_dir,'wl_ts.nc'))

fig, ax = plt.subplots(1,1)
idxs = da_wl.stations.values
for idx in idxs:
    da_wl.sel(stations = idx)['waterlevel'].plot(label = idx+1)

ax.set_title('')
ax.set_ylabel('Water level [m +MSL]')
ax.axhline(y = 0, color = 'darkgrey', ls = '--')
ax.legend(ncol = 4)

savefig(join(out_dir_his, 'forcing_wl.png'), dpi = 300,bbox_inches='tight')


In [ ]:
#Time series
import matplotlib.dates as mdates

fig, axs = plt.subplots(2,1)

ax = axs[0]
t = his_noveg['time']
z = his_noveg.values
z_veg = his_veg.values
elapsed_hours = (t.values - t.values[0]) / np.timedelta64(1, 'h')


ax.plot(elapsed_hours,z, color = 'navy', label = 'noveg')
ax.plot(elapsed_hours, z_veg, color = 'limegreen', label = 'veg')
ax.set_xticklabels([])
ax.set_ylabel('Water depth [m]')
ax.legend()

ax = axs[1]
delta = z_veg -z
ax.plot(elapsed_hours, delta)

ax.set_xlabel('Hours since start')
ax.set_ylabel('Water depth \ndifference [m]')

ax.set_ylim(-0.5,0.5)
ax.text(0.55, 0.2,'reduction due to veg', transform = ax.transAxes)
ax.text(0.55, 0.7,'increase due to veg', transform = ax.transAxes)
ax.axhline(0,ls = '--', color = 'black', lw = 0.5)


savefig(join(out_dir_his, 'obs_dif.png'), dpi = 300,bbox_inches='tight')

## Upload plots to EDITO storage

In [ ]:
from upload_model import upload_model_to_s3_bucket
upload_model_to_s3_bucket(join(sim_dir, 'figs'), 'SFINCS_OUTPUT')